In [45]:
import json
import scipy as sp
import scipy.sparse
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from numpy.random import default_rng


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.feature_selection import RFE
sns.set()
rng = default_rng()
%matplotlib inline

In [32]:
# hero_selection = sp.sparse.load_npz("../data/hero_selection.npz")
# match_winner = np.load("../data/match_winner.npy")
X = sp.sparse.load_npz("../data/big1_game_features.npz")
y = np.load("../data/big1_match_winners.npy")

y = (y * 2) - 1
print(X.shape, y.shape)

(2162047, 14161) (2162047,)


In [34]:
y.mean()

-0.00038435797186647656

In [35]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True) 

In [36]:
clf = LogisticRegression(random_state=21, max_iter=200).fit(x_train, y_train)
y_h = clf.predict(x_test)
acc = accuracy_score(y_test, y_h)

print(clf.score(x_test, y_test))


/home/jupyter-baoi/.conda/envs/dota/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5737548468660145


In [37]:
print(mean_squared_error(y_test, y_h))

1.704980612535942


In [38]:
print(r2_score(y_test, y_h))

-0.704981565835056


In [39]:
print(confusion_matrix(y_test, y_h))

[[186240 138310]
 [138159 185906]]


In [40]:
clf.coef_[0, 14]

0.0

In [41]:
y_test_prob = clf.predict_proba(x_test) 
m = y_test_prob.max(axis=1)

# antall matches som classifieren er trygg på.
# Hadde vi valgt ut kun disse gamesa, hadde vi nok hatt bedre acc
(m.mean() > m).sum() 

326404

In [46]:
def create_feature_index():
    heroes_path = "../data/hid_to_rid_dict.json"

    with open(heroes_path, 'r') as fp:
        heroes = json.load(fp)
        heroes = {int(k): int(v) for k, v in heroes.items()} 
        
    feature_set = set()
    for hero_id in heroes.keys():
        feature_set.add(("hero", hero_id))

        for hero_opp in heroes.keys():
            if hero_id == hero_opp:
                continue

            key = [hero_id, hero_opp]
            key.sort()
            feature_set.add(("pair_opp", tuple(key)))

        for hero_same in heroes.keys():
            if hero_id == hero_same:
                continue

            key = [hero_id, hero_same]
            key.sort()

            feature_set.add(("pair_same", tuple(key)))

    print("#feature_set:", len(feature_set))
    features_to_index = {}
    index_to_features = {}
    for k, v in enumerate(feature_set):
        features_to_index[v] = k
        index_to_features[k] = v
    
    return features_to_index, index_to_features
    
features_to_index, index_to_features = create_feature_index()

#feature_set: 14161


In [77]:
logreg = LogisticRegression(random_state=21, max_iter=200)
sel = RFE(logreg, n_features_to_select=5000, step=0.1, verbose=1)
sel = sel.fit(x_train, y_train)
order = sel.ranking_

feat_ranks = []
for i in order:
    feat_ranks.append(f"{i}: {index_to_features[i]}")
feat_ranks

Fitting estimator with 14161 features.
Fitting estimator with 12745 features.


KeyboardInterrupt: 